#### Data 612 - Project 2 : Content-Based and Collaborative Filtering<br>Date: June 18, 2019<br>Team Info: 
+ Christina Valore
+ Juliann McEachern 
+ Rajwant Mishra

<h1 align="center">Good Books Recommender System</h1>

## Dataset Selection

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load data from local csv  into pandas dataframe
book_tags = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/data/book_tags.csv')
tags = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/data/tags.csv')
books = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/data/books.csv')
ratings = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/data/ratings.tar.gz', compression='gzip')

In [29]:
book_tags.head()

,ratings.csv,user_id,book_id,rating
0,0.0,1.0,258.0,5.0
1,1.0,2.0,4081.0,4.0
2,2.0,2.0,260.0,5.0
3,3.0,2.0,9296.0,5.0
4,4.0,2.0,2318.0,3.0


In [3]:
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [10]:
np.shape(books)

(10000, 23)

In [32]:
np.shape(ratings)

(5976480, 4)

## Content-Based Filtering 

## User-User Collaborative Filtering 

## Item-Item Collaborative Filtering 

## Summary
Please provide at least one graph, and a textual summary of your findings and recommendations. 